In [1]:
import base64
from Crypto.Cipher import AES
from Crypto.Util.Padding import unpad,pad
import paho.mqtt.client as mqtt
import json
from tkinter import *
from tkinter import Tk, Label, Frame, Entry, Button, IntVar, Radiobutton, messagebox
from tkinter import Canvas, Tk, messagebox
from threading import Thread
import time
import math
from PIL import Image, ImageTk, ImageSequence
import matplotlib.pyplot as plt
from matplotlib.backends.backend_tkagg import FigureCanvasTkAgg

bpm_text_object = None
bpm_text = None
input_data = {}
bpm_label = None
bpm_values = []
heart_animation = None
plot_canvas = None
plot_ax = None 
# Decryption parameters
key = b'\xee\x16\xfb\xdf6|\xe8\xe4n\x0b\xeal\xb4\xf1\x05\xabk\xc0\xe8\x9a\xc2\xe0\xff8\x9e\xd8A\xa7d\xfbN\xf0'
iv = b'\xef?\x82\xb5\xa7\x89\x9b\xeet\xda\x8e\xaa\xc3\xd0{\xe4'
cipher = AES.new(key, AES.MODE_CBC, iv=iv)

key2 = b'\xee\x12\xfb\xdf6|\xe8\xe4n\x0b\xeal\xb4\xf1\x05\xabk\xc0\xe8\x9a\xc2\xe0\xff8\x9e\xd8A\xa7d\xfbN\xf0'
iv2 = b'\xef?\x81\xb5\xa7\x89\x9b\xeet\xda\x8e\xaa\xc3\xd0{\xe4'
cipher2 = AES.new(key2, AES.MODE_CBC, iv =iv2)

# MQTT parameters
broker_address = "localhost"
port = 1883
mqtt_topic = "bpm/data"
mqtt_topic2 = "data"
client2 = mqtt.Client("pub") 
client = mqtt.Client("Subscriber")
def publish_encrypted_data(data):
    try:
        # Encrypt the data
        ciphertext = cipher2.encrypt(pad(json.dumps(data).encode('utf-8'), AES.block_size))
        
        # Publish the encrypted data to MQTT
        client2.publish(mqtt_topic2, base64.b64encode(ciphertext))
        
        print(f"Original data: {data}")
        print(f"Encrypted data: {ciphertext}")
    except Exception as e:
        print(f"Error publishing data: {e}")


def submit():
    # Get the values from the radio buttons
    sex = sex_var.get()
    diet = diet_var.get()
    diabetes = diabetes_var.get()
    family_history = family_history_var.get()
    smoking = smoking_var.get()
    obesity = obesity_var.get()
    alcohol_consumption = alcohol_consumption_var.get()
    previous_heart_problems = previous_heart_problems_var.get()
    medication_use = medication_use_var.get()
    
    # Get the values from the entry widgets
    age = age_entry.get()
    income = income_entry.get()
    bmi = bmi_entry.get()
    
    try:
        age = int(age)
        if not 0 <= age <= 120:
            raise ValueError
    except ValueError:
        messagebox.showerror("Error", "Please enter a valid age (between 0 and 120)")
        return
    
    try:
        bmi = int(bmi)
        if not bmi >= 12:
            raise ValueError
    except ValueError:
        messagebox.showerror("Error", "Please enter a valid BMI")
        return
    
    try:
        income = int(income)
        if not income >= 0:
            raise ValueError
    except ValueError:
        messagebox.showerror("Error", "Please enter a valid Income")
        return
    
    # Create the input data dictionary
    input_data = {
        "age": age,
        "sex": sex,
        "diabetes": diabetes,
        "family_history": family_history,
        "smoking": smoking,
        "obesity": obesity,
        "alcohol_consumption": alcohol_consumption,
        "diet": diet,
        "previous_heart_problems": previous_heart_problems,
        "medication_use": medication_use,
        "income": income,
        "bmi": bmi
    }
    print(input_data)
       
    client2.connect(broker_address, port) 
    publish_encrypted_data(input_data)
    client2.disconnect()  
    
    root.destroy()
    create_new_window()
    

def create_new_window():
    new_window = Tk()
    new_window.title("Heart Rate Monitor")
    global bpm_label
    global bpm_values
    global heart_animation
    global plot_canvas
    global plot_ax
    new_window.configure(bg="white")
    
    # Load heart animation GIF
    heart_animation = Image.open("heart.png")
    heart_animation.thumbnail((332.75, 290.75))  # Resize the heart image
    heart_image = ImageTk.PhotoImage(heart_animation)

    # Canvas to display heart animation
    heart_canvas = Canvas(new_window, bg="white", width=332.75, height=290.75, highlightthickness=0)
    heart_canvas.pack()

    # Function to pulsate the heart image
    def pulsate(size, direction=1):
        heart_canvas.delete("heart")
        new_size = size + (5 * direction)
        # Calculate the coordinates for centering the image
        x_center = (332.75 - new_size) / 2
        y_center = (290.75 - new_size) / 2
        heart_animation_resized = heart_animation.resize((new_size, new_size))
        heart_image_resized = ImageTk.PhotoImage(heart_animation_resized)
        heart_canvas.create_image(x_center, y_center, image=heart_image_resized, tags="heart", anchor=NW)
        heart_canvas.image = heart_image_resized
        new_window.after(100, pulsate, new_size, -direction)  # Change direction after each cycle

    # Start pulsating animation
    pulsate(100)
    
    # Label to display BPM value
    bpm_label = Label(new_window, text="", font=("Helvetica", 20), fg="black", bg="white")
    bpm_label.place(relx=0.5, rely=0.05, anchor=CENTER)

    # Initialize list to store BPM values
    bpm_values = []


    # Callback function to display the BPM value and update the graph
    def on_message(client, userdata, message):
        global bpm_label
        global bpm_values

        try:
            print("Message Received")
            # Decode the received message and extract the BPM value
            ciphertext = base64.b64decode(message.payload)
            decrypted_data = unpad(cipher.decrypt(ciphertext), AES.block_size)
        
            json_data = json.loads(decrypted_data.decode('utf-8'))
        
            y_predict = json_data['y_predict']
            bpm_value = json_data['sensor_data']
        
            # Update the BPM value label
            bpm_label.config(text=f"BPM: {int(bpm_value)}")

            # Add the BPM value to the list of values
            bpm_values.append(int(bpm_value))  # Convert bpm_value to float

            # Keep only the last 15 BPM values
            if len(bpm_values) > 15:
                bpm_values = bpm_values[-15:]

            print("BPM Values:", bpm_values)  # Debugging

            if len(bpm_values) == 1:
                create_plot_canvas(new_window)

            # Update the plot
            update_plot()

            # Check if y_predict indicates heart risk
            if y_predict == 1:
                messagebox.showwarning("Heart Risk", "You have a heart risk. Please consult a doctor.")
        except Exception as e:
            print(f"Error processing message: {e}")
            
    def update_plot():
        global bpm_values
        global plot_ax

        plot_ax.clear()
        plot_ax.plot(range(len(bpm_values)), bpm_values, marker='o', color='blue')
        plot_ax.set_xlabel('')
        plot_ax.set_ylabel('BPM')
        plot_ax.set_title('Heart Rate Monitor')
        plot_ax.grid(True)
        # Set y-axis limits dynamically based on the BPM values
        if len(bpm_values) > 0:
            min_bpm = min(bpm_values) - 5
            max_bpm = max(bpm_values) + 5
            plot_ax.set_ylim(min_bpm, max_bpm)
        # Set x-axis limits to ensure all data points are visible
        plot_ax.set_xlim(-0.5, len(bpm_values) - 0.5)
        plt.tight_layout()
        plot_canvas.draw()

    def create_plot_canvas(window):
        global plot_canvas
        global plot_ax

        # Create plot
        fig, plot_ax = plt.subplots()
        fig.patch.set_facecolor('white')  # Set background of plot to white
        plot_canvas = FigureCanvasTkAgg(fig, master=window)
        plot_canvas.get_tk_widget().pack(side=TOP, fill=BOTH, expand=1)
            
    client.on_message = on_message

    try:
        # Connect to the MQTT Broker
        client.connect(broker_address, port)
        print("Connected to MQTT broker")

        # Subscribe to the MQTT topic
        client.subscribe(mqtt_topic)
        print(f"Subscribed to topic: {mqtt_topic}")

        # Set the on_message callback
        client.on_message = on_message
        print("on_message callback registered")

        # Start the MQTT loop to listen for messages
        client.loop_start()
    except Exception as e:
        print(f"Error: {e}")

    new_window.mainloop()


root = Tk()

myLabel = Label(root, text="Personal Heart Monitor")
myLabel.pack(pady=10)
myLabel2 = Label(root, text="All inputs must be in digits and positive")
myLabel2.pack()
fields_frame = Frame(root)
fields_frame.pack(padx=10, pady=10)

labels = [
    "Age:",
    "Enter income:",
    "Enter bmi:",
    "Sex(female-0, male-1):",
    "Do you have diabetes:",
    "Do you have any family history of heart problems:",
    "Do you smoke:",
    "Are you obese:",
    "Do you consume alcohol:",
    "Did you have any heart problems previously:",
    "Do you use any medication for heart problem:",
    "How is your diet (healthy-1, unhealthy-2, average-0):"
  
]

entry_widgets = []
for i, label_text in enumerate(labels):
    label = Label(fields_frame, text=label_text)
    label.grid(row=i, column=0, sticky="w")
    if i <= 2:
        entry_widget = Entry(fields_frame)
        entry_widget.grid(row=i, column=1, sticky="w")
        entry_widgets.append(entry_widget)

age_entry = entry_widgets[0]
income_entry = entry_widgets[1]
bmi_entry = entry_widgets[2]

# Radio buttons for Sex
sex_var = IntVar()
sex_buttons = [("Female", 0), ("Male", 1)]
for i, (text, value) in enumerate(sex_buttons):
    Radiobutton(fields_frame, text=text, variable=sex_var, value=value).grid(row=3, column=1+i, sticky="w")

# Radio buttons for Diet
diet_var = IntVar()
diet_buttons = [("Healthy", 1), ("Unhealthy", 2), ("Average", 0)]
for i, (text, value) in enumerate(diet_buttons):
    Radiobutton(fields_frame, text=text, variable=diet_var, value=value).grid(row=11, column=1+i, sticky="w")

# Radio buttons for Yes/No options
diabetes_var = IntVar()
family_history_var = IntVar()
smoking_var = IntVar()
obesity_var = IntVar()
alcohol_consumption_var = IntVar()
previous_heart_problems_var = IntVar()
medication_use_var = IntVar()

radio_vars = [
    diabetes_var,
    family_history_var,
    smoking_var,
    obesity_var,
    alcohol_consumption_var,
    previous_heart_problems_var,
    medication_use_var
]

radio_buttons = [("Yes", 1), ("No", 0)]
for row_num, radio_var in enumerate(radio_vars, start=4):
    for i, (text, value) in enumerate(radio_buttons):
        Radiobutton(fields_frame, text=text, variable=radio_var, value=value).grid(row=row_num, column=2*i+1, sticky="w")

# Submit button
submit_button = Button(root, text="Submit", command=submit)
submit_button.pack(pady=10)

root.mainloop()


{'age': 22, 'sex': 0, 'diabetes': 0, 'family_history': 0, 'smoking': 0, 'obesity': 0, 'alcohol_consumption': 0, 'diet': 0, 'previous_heart_problems': 0, 'medication_use': 0, 'income': 22000, 'bmi': 21}
Original data: {'age': 22, 'sex': 0, 'diabetes': 0, 'family_history': 0, 'smoking': 0, 'obesity': 0, 'alcohol_consumption': 0, 'diet': 0, 'previous_heart_problems': 0, 'medication_use': 0, 'income': 22000, 'bmi': 21}
Encrypted data: b'\x16Kw\x12\xa0\xf2\xb9\xcaY\x86U\xa4\xa1\xc0\xe6\xe2$\xe8h\xf1\xd3\xd1\x81\xcfk\x06\x0bd\x7f\x91\x7f\xd3\x08\xdb"\x08\x9d\x9a\xcaY\x06\x1f$\xde\x18\xc2$l\xf4\x18\xeb\xd1K\xe4\x05\x05\xc2\xfe\xda\r\xe7N\xbd\xceg;\x1f\x1d$\xaf\xc0\xe1F\xbc\x91\xc1~\x01\x94(\xe5 w\xc7\xf3:^\xd21\xfd>\xd2%e\xc5\xb6a\x7f\x846\xe3\x8a\xee\xd881\xd1\xa6E\x99\x1c\xaflG\xb8\xbe\xab\xfb\xa0\xe37\x058\xe8q\xd9\x1a@\x8b\x81\x0cLyL\x82\xb7\xdf\xc4\xee~-\xa6\x81\x88\xd6\xf0\xca{\xc2\xc7\x88\xd8\xc1M\x1e\x84`\x96\xa5a\xd8x\xdb\xe7WS\xb0\x91\xf5\xf1\xd9\xda\xc2\xd3E\x99\xe7\x91\xbb\x11\tm\